In [1]:
%matplotlib notebook
import matplotlib as mpl
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import numpy as np
import math
import controls as ctrl
import pandas as pd
import os

**Programmer:** Marco Gutierrez

# Defining our endowments

In [7]:
original_task_endowments = ctrl.task_endowments
total_endowment = sum(original_task_endowments)
unique_task_endowments = list(set(ctrl.task_endowments))
unique_task_endowments.sort()

# Defining Payoff Functions

## 1. Public Payoff

### Parameters

In [17]:
theta = 0.025 # exponential parameter

### Payoff Function

In [13]:
def public_payoff(total_endowment, tax_rate):
    """
    This function calculates the public payoff for the players using
    the Kamei (2019) functions
    
    Input: Theta (exponential parameter), the sum of all the endowments (total_endowment) and the Tax rate (tax_rate)
    Output: Public payoff for each player
    """
    total_public_contribution = tax_rate*total_endowment
    
    if total_public_contribution <= 192:
        # public income for current total contribution
        return 101 / (1 + 100 * math.exp(-theta * total_public_contribution)) - 1
    else:
        return 101 / (1 + 100 * math.exp(-theta * 192)) - 1

## 2. Private Payoff

### Parameters

In [11]:
gamma = 0.85 # Degree of private payoff function
alpha = ctrl.alpha # Private productivity additive parameter
beta = ctrl.beta # Private productivity multiplicative parameter

### Private Productivity Function

In [16]:
def private_productivity(alpha, beta, tax_rate, total_endowment):
    """
    Determines the productivity of private payoffs.
    
    Input: Function base parameters (alpha, beta), the current tax rate and the sum of all the endowments (total_endowment)
    Output: Private productivity
    """
    total_public_contribution = tax_rate*total_endowment
    
    if total_public_contribution <= 192:
        return ctrl.alpha + ctrl.beta*total_public_contribution
    else:
        return ctrl.alpha + ctrl.beta*192

### Payoff Function

In [19]:
def private_payoff(private_productivity, gamma, tax_rate, endowment):
    """
    Determines the private payoff for each player.
    
    Input: Private productivity, degree of private payoff function (gamma), current tax rate and endowment
    Output: Private payoff
    """
    private_budget = (1-tax_rate)*endowment
    
    return (private_productivity*private_budget)**gamma